<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Exploratory Data Analysis with SQL
Estimated time needed: 60 minutes


# Introduction
Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection
Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.  
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [1]:
# import library
library(RODBC)

In [2]:
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            # e.g. "bludb"
dsn_hostname <- "0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" # e.g "54a2f15b-5c0f-46df-8954-.databases.appdomain.cloud"
dsn_port <- "31249"   # e.g. "32733" 
dsn_protocol <- "TCPIP"            # i.e. "TCPIP"
dsn_uid <- "lpg49402"        # e.g. "zjh17769"
dsn_pwd <- "r45AtWNlAlu2bYaH"      # e.g. "zcwd4+8gbq9bm5k4"  
dsn_security <- "ssl"

In [3]:
# provide your solution here
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path)
conn

RODBC Connection 1
Details:
  case=nochange
  DRIVER={IBM DB2 ODBC DRIVER}
  UID=lpg49402
  PWD=******
  DATABASE=bludb
  HOSTNAME=0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud
  PORT=31249
  PROTOCOL=TCPIP
  SECURITY=SSL


## Task 1 - Record Count
#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [20]:
# provide your solution here
query1 <- "SELECT COUNT(*) records FROM seoul_bike_sharing;"
sqlQuery(conn, query1, believeNRows=FALSE)

,RECORDS
,<int>
1,8465


## Task 2 - Operational Hours
#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [21]:
# provide your solution here
query2 <- "SELECT COUNT(DISTINCT hour) num_hours
            FROM seoul_bike_sharing
            WHERE rented_bike_count > 0;"
sqlQuery(conn, query2, believeNRows=FALSE)

,NUM_HOURS
,<int>
1,24


## Task 3 - Weather Outlook
#### Query the the weather forecast for Seoul over the next 3 hours.
Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [22]:
# provide your solution here
query3 <- "SELECT *
        FROM cities_weather_forecast
        WHERE city = 'Seoul' AND forecast_datetime >= '2021-04-16 12:00:00' AND forecast_datetime < '2021-04-16 15:00:00'
        ORDER BY forecast_datetime ASC
        LIMIT 1;"
sqlQuery(conn, query3, believeNRows=FALSE)

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


## Task 4 - Seasons
#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [23]:
# provide your solution here
query4 <- "SELECT DISTINCT seasons
FROM seoul_bike_sharing;"
sqlQuery(conn, query4, believeNRows=FALSE)

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range
#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [24]:
# provide your solution here
query5 <- "SELECT MAX(date) first_date, MIN(date) last_date
FROM seoul_bike_sharing;"
sqlQuery(conn, query5, believeNRows=FALSE)

,FIRST_DATE,LAST_DATE
,<fct>,<fct>
1,31/12/2017,01/01/2018


## Task 6 - Subquery - 'all-time high'
#### determine which date and hour had the most bike rentals.


### Solution 6


In [28]:
# provide your solution here
query6 <- "SELECT date, hour, MAX(rented_bike_count) bike_count
            FROM seoul_bike_sharing
            GROUP BY date, hour
            ORDER BY MAX(rented_bike_count) DESC
            LIMIT 1;"
sqlQuery(conn, query6, believeNRows=FALSE)

,DATE,HOUR,BIKE_COUNT
,<fct>,<int>,<int>
1,19/06/2018,18,3556


## Task 7 - Hourly popularity and temperature by season
#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [29]:
# provide your solution here
query7 <- "SELECT seasons, hour, AVG(temperature) avg_temperature, AVG(rented_bike_count) avg_bike_count
            FROM seoul_bike_sharing
            GROUP BY seasons, hour
            ORDER BY avg_bike_count DESC
            LIMIT 10;"
sqlQuery(conn, query7, believeNRows=FALSE)

,SEASONS,HOUR,AVG_TEMPERATURE,AVG_BIKE_COUNT
,<fct>,<int>,<dbl>,<int>
1,Summer,18,29.38696,2135
2,Autumn,18,16.03086,1983
3,Summer,19,28.27283,1889
4,Summer,20,27.06630,1801
5,Summer,21,26.27826,1754
6,Spring,18,15.97222,1689
7,Summer,22,25.69891,1567
8,Autumn,17,17.27778,1562
9,Summer,17,30.07500,1526


## Task 8 - Rental Seasonality
#### Find the average hourly bike count during each season. 
Also include the minimum, maximum, and standard deviation of the hourly bike count for each season. 


### Solution 8


In [31]:
# provide your solution here
query8 <- "SELECT seasons, hour, AVG(rented_bike_count) avg_bike_count, MIN(rented_bike_count) min_bike_count, MAX(rented_bike_count) max_bike_count, STDDEV_POP(rented_bike_count) std_bike_count
            FROM seoul_bike_sharing
            GROUP BY seasons, hour
            ORDER BY seasons, hour;"
sqlQuery(conn, query8, believeNRows=FALSE)

,SEASONS,HOUR,AVG_BIKE_COUNT,MIN_BIKE_COUNT,MAX_BIKE_COUNT,STD_BIKE_COUNT
,<fct>,<int>,<int>,<int>,<int>,<dbl>
1,Autumn,0,709,119,1336,219.14298
2,Autumn,1,552,144,1001,191.54216
3,Autumn,2,377,55,785,144.90134
4,Autumn,3,256,28,514,102.53108
5,Autumn,4,169,24,338,58.63957
6,Autumn,5,163,24,264,53.88174
7,Autumn,6,359,23,691,180.27049
8,Autumn,7,788,5,1556,457.96861
9,Autumn,8,1345,6,2391,758.35956


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality
#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?
Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all. 


### Solution 9


In [33]:
# provide your solution here
query9 <- "SELECT seasons,
            AVG(temperature) avg_temperature,
            AVG(humidity) avg_humidity,
            AVG(wind_speed) avg_wind_speed,
            AVG(visibility) avg_visibility,
            AVG(dew_point_temperature) avg_dew_point_temperature,
            AVG(solar_radiation) avg_solar_radiation,
            AVG(rainfall) avg_rainfall,
            AVG(snowfall) avg_snowfall,
            AVG(rented_bike_count) avg_bike_count
            FROM seoul_bike_sharing
            GROUP BY seasons
            ORDER BY avg_bike_count DESC;"
sqlQuery(conn, query9, believeNRows=FALSE)

,SEASONS,AVG_TEMPERATURE,AVG_HUMIDITY,AVG_WIND_SPEED,AVG_VISIBILITY,AVG_DEW_POINT_TEMPERATURE,AVG_SOLAR_RADIATION,AVG_RAINFALL,AVG_SNOWFALL,AVG_BIKE_COUNT
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Summer,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000,1034
2,Autumn,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026,924
3,Spring,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000,746
4,Winter,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000,225


## Task 10 - Total Bike Count and City Info for Seoul
#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.
Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [51]:
# provide your solution here
query10 <- "SELECT
            wc.city,
            wc.country,
            wc.lat,
            wc.lng,
            wc.population,
            SUM(bs.bicycles) AS total_bikes
            FROM
            world_cities wc
            JOIN bike_sharing_systems bs ON wc.CITY_A = bs.CITY
            WHERE
            wc.CITY = 'Seoul'
            GROUP BY
            wc.CITY,
            wc.COUNTRY,
            wc.LAT,
            wc.LNG,
            wc.POPULATION;"
sqlQuery(conn, query10, believeNRows=FALSE)

,CITY,COUNTRY,LAT,LNG,POPULATION,TOTAL_BIKES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Seoul,"Korea, South",37.58,127,21794000,20000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system
#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.
Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [52]:
# provide your solution here
query11 <- "SELECT wc.CITY, wc.COUNTRY, wc.LAT, wc.LNG, wc.POPULATION, bs.BICYCLES
            FROM WORLD_CITIES wc
            INNER JOIN BIKE_SHARING_SYSTEM bs ON WC.CITY_ASCII = bs.CITY
            WHERE bs.BICYCLES >= 15000 AND bs.BICYCLES <= 20000;"
            sqlQuery(conn, query10, believeNRows=FALSE)

,CITY,COUNTRY,LAT,LNG,POPULATION,TOTAL_BIKES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Seoul,"Korea, South",37.58,127,21794000,20000


In [53]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                |
| ---------- | ------- | ------------- | --------------------------------------------------------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                               |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review                               |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins                                  |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation                                                          |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
